In [ ]:
from jupyter_plotly_dash import JupyterDash

import dash
import dash_leaflet as dl
import dash_core_components as dcc
import dash_html_components as html
import plotly.express as px
import dash_table as dt
from dash.dependencies import Input, Output, State

import os
import numpy as np
import pandas as pd
from pymongo import MongoClient
from bson.json_util import dumps
import base64

import AnimalShelter

# Importing the AnimalShelter module 
from AnimalShelter import AnimalShelter



###########################
# Data Manipulation / Model
###########################
username = "dmhunter"
password = "Devdarb 3358!"
# Instatiating Animalshelter
shelter = AnimalShelter(username, password)

# Class read method must support return of cursor object 
df = pd.DataFrame.from_records(shelter.read({}))



#########################
# Dashboard Layout / View
#########################
# create dash application

external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']
app = dash.Dash(__name__, external_stylesheets=external_stylesheets)

# Appropriate Logo
image_filename = 'GSLogo.png'
encoded_image = base64.b64encode(open(image_filename, 'rb').read())
                               
app.layout = html.Div([
    html.Div(id='hidden-div', style={'display':'none'}),
        # Anchor tag w/ clickable image
        html.A([
            html.Img(
                src='data:image/png;base64,{}'.format(encoded_image.decode()),
                # Styling
                style={
                    'height' : '10%',
                    'width' : '10%',
                    'float' : 'left',
                    'position' : 'relative',
                })
        ], href='https://www.snhu.edu'),
        html.Center(html.B(html.H1("Devin's Shelter Dashboard", style={'padding-top' : '115px'}))),
    html.Hr(),
    html.Div(      
        # Code for dropdown menu
        dcc.Dropdown(
            id='demo-dropdown',
            options=[
                {'label': 'All', 'value': 'All'},
                {'label': 'Water Rescue', 'value': 'WR'},
                {'label': 'Disaster Rescue/Individual Tracking', 'value': 'DR/IT'},
                {'label': 'Mountain/Wilderness Rescue', 'value': 'M/WR'},           
            ],
            value='All'
        ),
        
    ),
    html.Hr(),
    dt.DataTable(
        id='datatable-id',
        columns=[
            {"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns
        ],
        data=df.to_dict('records'),
        # Allows 10 rows per page. 
        editable=False,
        sort_action="native",
        sort_mode="multi",
        column_selectable=True,
        row_selectable=True,
        row_deletable=False,
        selected_columns=[],
        selected_rows=[0],
        page_action="native",
        page_current= 0,
        page_size= 10,
    ),
    html.Br(),
    html.Hr(),
    # This sets up the dashboard so that your chart and your geolocation chart are side-by-side
    html.Div(className='row',
         style={'display' : 'flex'},
             children=[
        html.Div(
            id='map-id',
            className='col s12 m6',
            ),
        html.Div(
            id='graph-id',
            className='col s12 m6',
            )
        ])
])

#############################################
# Interaction Between Components / Controller
#############################################



@app.callback([Output('datatable-id','data'),
               Output('datatable-id','columns')],
              [Input('demo-dropdown', 'value')])
def update_dashboard(value):
# Code to filter interactive data table with MongoDB queries
    if (value == 'All'):
        df = pd.DataFrame.from_records(shelter.read({}))
    elif (value == 'WR'):
        df = pd.DataFrame(list(shelter.read({"breed" : {"$in": ["Labrador Retriever Mix", "Chesapeake Bay Retriever", "Newfoundland"]}, "sex_upon_outcome" : "Intact Female", "age_upon_outcome_in_weeks": {"$lte": 156, "$gte": 26}})))
    elif (value == 'M/WR'):
        df = pd.DataFrame(list(shelter.read({"breed" : {"$in": ["German Shephard", "Alaskan Malamute", "Old English Sheepdog", "Siberian Husky", "Rottweiler"]}, "sex_upon_outcome" : "Intact Male", "age_upon_outcome_in_weeks": {"$lte": 156, "$gte": 26}})))
    elif (value == 'DR/IT'):
        df = pd.DataFrame(list(shelter.read({"breed" : {"$in": ["Doberman Pinscher", "German Shephard", "Golden Retriever", "Bloodhound", "Rottweiler"]}, "sex_upon_outcome" : "Intact Male", "age_upon_outcome_in_weeks": {"$lte": 300, "$gte": 20}})))

        
    columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns]
    data=df.to_dict('records')
        
        
    return (data,columns)


# Changes the background color of selected columns
@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    return [{
        'if': { 'column_id': i },
        'background_color': '#D2F3FF'
    } for i in selected_columns]

# Chart
@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "derived_viewport_data"),
    ])
def update_graphs(viewData):
    dff = pd.DataFrame.from_dict(viewData)
    
    return [
        dcc.Graph(            
            figure = px.pie(dff, 
                            values='age_upon_outcome_in_weeks', 
                            names='breed',
                            title='Available Dogs Sorted by Breed')
        )    
    ]

# Map that changes based on selection
@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_viewport_data"),
     Input('datatable-id', 'derived_viewport_selected_rows')])
def update_map(viewData, row_ids):
    dff = pd.DataFrame.from_dict(viewData)
    # Sets first num in row_ids array to the last selected row ID
    row_ids[0] = row_ids[len(row_ids) - 1]
    return [
        # row_ids[0] gets last selected row from DataTable and parameters 13 and 14 are long and lat of rows
        dl.Map(style={'width': '1000px', 'height': '500px'}, center=[dff.iloc[row_ids[0],13],dff.iloc[row_ids[0],14]], zoom=10, children=[
            dl.TileLayer(id="base-layer-id"),
            # Marker with tool tip and popup
            dl.Marker(position=[dff.iloc[row_ids[0],13],dff.iloc[row_ids[0],14]], children=[
                # parameter num 4 gives the breed of animal
                dl.Tooltip(dff.iloc[row_ids[0],4]),
                dl.Popup([
                    html.H1("Animal Name"),
                    # parameter 9 gives animal's name
                    html.P(dff.iloc[row_ids[0],9])
                ])
            ])
        ])
    ]


if __name__ == '__main__':
    app.run_server(debug=False)

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
127.0.0.1 - - [15/Apr/2023 17:17:35] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [15/Apr/2023 17:17:36] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [15/Apr/2023 17:17:36] "GET /_favicon.ico?v=1.10.0 HTTP/1.1" 200 -
127.0.0.1 - - [15/Apr/2023 17:17:36] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [15/Apr/2023 17:17:37] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [15/Apr/2023 17:17:37] "POST /_dash-update-component HTTP/1.1" 200 -


Exception on /_dash-update-component [POST]
Traceback (most recent call last):
  File "/usr/local/anaconda/lib/python3.6/site-packages/flask/app.py", line 2446, in wsgi_app
    response = self.full_dispatch_request()
  File "/usr/local/anaconda/lib/python3.6/site-packages/flask/app.py", line 1951, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "/usr/local/anaconda/lib/python3.6/site-packages/flask/app.py", line 1820, in handle_user_exception
    reraise(exc_type, exc_value, tb)
  File "/usr/local/anaconda/lib/python3.6/site-packages/flask/_compat.py", line 39, in reraise
    raise value
  File "/usr/local/anaconda/lib/python3.6/site-packages/flask/app.py", line 1949, in full_dispatch_request
    rv = self.dispatch_request()
  File "/usr/local/anaconda/lib/python3.6/site-packages/flask/app.py", line 1935, in dispatch_request
    return self.view_functions[rule.endpoint](**req.view_args)
  File "/usr/local/anaconda/lib/python3.6/site-packages/dash/dash.py", line 1

127.0.0.1 - - [15/Apr/2023 17:17:37] "POST /_dash-update-component HTTP/1.1" 500 -
127.0.0.1 - - [15/Apr/2023 17:17:37] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [15/Apr/2023 17:17:37] "POST /_dash-update-component HTTP/1.1" 200 -


Exception on /_dash-update-component [POST]
Traceback (most recent call last):
  File "/usr/local/anaconda/lib/python3.6/site-packages/flask/app.py", line 2446, in wsgi_app
    response = self.full_dispatch_request()
  File "/usr/local/anaconda/lib/python3.6/site-packages/flask/app.py", line 1951, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "/usr/local/anaconda/lib/python3.6/site-packages/flask/app.py", line 1820, in handle_user_exception
    reraise(exc_type, exc_value, tb)
  File "/usr/local/anaconda/lib/python3.6/site-packages/flask/_compat.py", line 39, in reraise
    raise value
  File "/usr/local/anaconda/lib/python3.6/site-packages/flask/app.py", line 1949, in full_dispatch_request
    rv = self.dispatch_request()
  File "/usr/local/anaconda/lib/python3.6/site-packages/flask/app.py", line 1935, in dispatch_request
    return self.view_functions[rule.endpoint](**req.view_args)
  File "/usr/local/anaconda/lib/python3.6/site-packages/dash/dash.py", line 1

127.0.0.1 - - [15/Apr/2023 17:17:38] "POST /_dash-update-component HTTP/1.1" 500 -
127.0.0.1 - - [15/Apr/2023 17:17:38] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [15/Apr/2023 17:17:38] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [15/Apr/2023 17:17:41] "GET /_favicon.ico?v=1.10.0 HTTP/1.1" 200 -


Exception on /_dash-update-component [POST]
Traceback (most recent call last):
  File "/usr/local/anaconda/lib/python3.6/site-packages/flask/app.py", line 2446, in wsgi_app
    response = self.full_dispatch_request()
  File "/usr/local/anaconda/lib/python3.6/site-packages/flask/app.py", line 1951, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "/usr/local/anaconda/lib/python3.6/site-packages/flask/app.py", line 1820, in handle_user_exception
    reraise(exc_type, exc_value, tb)
  File "/usr/local/anaconda/lib/python3.6/site-packages/flask/_compat.py", line 39, in reraise
    raise value
  File "/usr/local/anaconda/lib/python3.6/site-packages/flask/app.py", line 1949, in full_dispatch_request
    rv = self.dispatch_request()
  File "/usr/local/anaconda/lib/python3.6/site-packages/flask/app.py", line 1935, in dispatch_request
    return self.view_functions[rule.endpoint](**req.view_args)
  File "/usr/local/anaconda/lib/python3.6/site-packages/dash/dash.py", line 1

127.0.0.1 - - [15/Apr/2023 17:17:42] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [15/Apr/2023 17:17:42] "POST /_dash-update-component HTTP/1.1" 500 -


Exception on /_dash-update-component [POST]
Traceback (most recent call last):
  File "/usr/local/anaconda/lib/python3.6/site-packages/flask/app.py", line 2446, in wsgi_app
    response = self.full_dispatch_request()
  File "/usr/local/anaconda/lib/python3.6/site-packages/flask/app.py", line 1951, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "/usr/local/anaconda/lib/python3.6/site-packages/flask/app.py", line 1820, in handle_user_exception
    reraise(exc_type, exc_value, tb)
  File "/usr/local/anaconda/lib/python3.6/site-packages/flask/_compat.py", line 39, in reraise
    raise value
  File "/usr/local/anaconda/lib/python3.6/site-packages/flask/app.py", line 1949, in full_dispatch_request
    rv = self.dispatch_request()
  File "/usr/local/anaconda/lib/python3.6/site-packages/flask/app.py", line 1935, in dispatch_request
    return self.view_functions[rule.endpoint](**req.view_args)
  File "/usr/local/anaconda/lib/python3.6/site-packages/dash/dash.py", line 1

127.0.0.1 - - [15/Apr/2023 17:17:42] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [15/Apr/2023 17:17:42] "POST /_dash-update-component HTTP/1.1" 500 -
127.0.0.1 - - [15/Apr/2023 17:17:42] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [15/Apr/2023 17:17:42] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [15/Apr/2023 17:17:42] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [15/Apr/2023 17:17:42] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [15/Apr/2023 17:18:07] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [15/Apr/2023 17:18:07] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [15/Apr/2023 17:18:07] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [15/Apr/2023 17:18:10] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [15/Apr/2023 17:18:10] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [15/Apr/2023 17:18:10] "POST /_dash-update-component HTTP/1.1" 200 -
